In [31]:
from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

from finbert import *
import utils as tools

%load_ext autoreload
%autoreload 2

# project_dir = Path.cwd().parent
project_dir = Path.cwd()
pd.set_option('max_colwidth', -1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Dataset 
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split
import os


# if not os.path.exists('data/sentiment_data'):
#    os.makedirs('data/sentiment_data')

# parser = argparse.ArgumentParser(description='Sentiment analyzer')
#parser.add_argument('--data_path', type=str, help='Path to the text file.')

#args = parser.parse_args()
data = pd.read_csv('./finbert_sentiment_data/Sentences_50Agree.txt', sep='.@', names=['text','label'])

train, test = train_test_split(data, test_size=0.2, random_state=0)
train, valid = train_test_split(train, test_size=0.1, random_state=0)

train.to_csv('finbert_sentiment_data/train.csv',sep='\t')
test.to_csv('finbert_sentiment_data/test.csv',sep='\t')
valid.to_csv('finbert_sentiment_data/validation.csv',sep='\t')

/home/drdata/anaconda3/envs/keras_dev/lib/python3.6/site-packages/ipykernel_launcher.py:15: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


In [29]:
Path.cwd()

PosixPath('/home/drdata/finbert')

In [28]:
project_dir

PosixPath('/home/drdata')

In [6]:

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

In [34]:

lm_path = './models/TRC2'
cl_path = './models/classifier_model/finbert-sentiment'
cl_data_path = './finbert_sentiment_data'

In [35]:
try:
    shutil.rmtree(cl_path) 
except:
    pass

bertmodel = AutoModelForSequenceClassification.from_pretrained(lm_path,cache_dir=None, num_labels=3)


config = Config(   data_dir=cl_data_path,
                   bert_model=bertmodel,
                   num_train_epochs=4,
                   model_dir=cl_path,
                   max_seq_length = 48,
                   train_batch_size = 32,
                   learning_rate = 2e-5,
                   output_mode='classification',
                   warm_up_proportion=0.2,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=True)

Some weights of the model checkpoint at ./models/TRC2 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./m

In [36]:

finbert = FinBert(config)
finbert.base_model = 'bert-base-uncased'
finbert.config.discriminate=True
finbert.config.gradual_unfreeze=True

In [37]:
finbert.prepare_model(label_list=['positive','negative','neutral'])


04/22/2021 21:54:43 - INFO - finbert -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False


In [38]:
# Get the training examples
train_data = finbert.get_data('train')

In [39]:
model = finbert.create_the_model()


In [40]:

# This is for fine-tuning a subset of the model.

freeze = 6

for param in model.bert.embeddings.parameters():
    param.requires_grad = False
    
for i in range(freeze):
    for param in model.bert.encoder.layer[i].parameters():
        param.requires_grad = False

In [43]:
trained_model = finbert.train(train_examples = train_data, model = model)


04/22/2021 22:14:28 - INFO - utils -   *** Example ***
04/22/2021 22:14:28 - INFO - utils -   guid: train-1
04/22/2021 22:14:28 - INFO - utils -   tokens: [CLS] after the reporting period , bio ##tie north american licensing partner so ##max ##on pharmaceuticals announced positive results with na ##lm ##efe ##ne in a pilot phase 2 clinical trial for smoking ce ##ssa ##tion [SEP]
04/22/2021 22:14:28 - INFO - utils -   input_ids: 101 2044 1996 7316 2558 1010 16012 9515 2167 2137 13202 4256 2061 17848 2239 24797 2623 3893 3463 2007 6583 13728 27235 2638 1999 1037 4405 4403 1016 6612 3979 2005 9422 8292 11488 3508 102 0 0 0 0 0 0 0 0 0 0 0
04/22/2021 22:14:28 - INFO - utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
04/22/2021 22:14:28 - INFO - utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/22/2021 22:14:28 - INFO - utils -   label: positive

04/22/2021 22:14:34 - INFO - utils -   *** Example ***
04/22/2021 22:14:34 - INFO - utils -   guid: validation-1
04/22/2021 22:14:34 - INFO - utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
04/22/2021 22:14:34 - INFO - utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/22/2021 22:14:34 - INFO - utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/22/2021 22:14:34 - INFO - utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/22/2021 22:14:34 - INFO - utils -   label: neutral (id = 2)
04/22/2021 22:14:34 - INFO - finbert -   ***** Loading data *****
04/22/2021 22:14:34 - INFO - finbert -     Num examples = 388
04/22/202


Validation losses: [0.3559717031625601]
No best model found
4
./models/classifier_model/finbert-sentiment


Epoch:  25%|██▌       | 1/4 [00:07<00:22,  7.61s/it]

04/22/2021 22:14:44 - INFO - utils -   *** Example ***
04/22/2021 22:14:44 - INFO - utils -   guid: validation-1
04/22/2021 22:14:44 - INFO - utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
04/22/2021 22:14:44 - INFO - utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/22/2021 22:14:44 - INFO - utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/22/2021 22:14:44 - INFO - utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/22/2021 22:14:44 - INFO - utils -   label: neutral (id = 2)
04/22/2021 22:14:44 - INFO - finbert -   ***** Loading data *****
04/22/2021 22:14:44 - INFO - finbert -     Num examples = 388
04/22/202

...


Validation losses: [0.3559717031625601, 0.3559717031625601, 0.3559717031625601]


Epoch:  75%|███████▌  | 3/4 [00:28<00:09,  9.07s/it]

04/22/2021 22:15:07 - INFO - utils -   *** Example ***
04/22/2021 22:15:07 - INFO - utils -   guid: validation-1
04/22/2021 22:15:07 - INFO - utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
04/22/2021 22:15:07 - INFO - utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/22/2021 22:15:07 - INFO - utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/22/2021 22:15:07 - INFO - utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/22/2021 22:15:07 - INFO - utils -   label: neutral (id = 2)
04/22/2021 22:15:07 - INFO - finbert -   ***** Loading data *****
04/22/2021 22:15:07 - INFO - finbert -     Num examples = 388
04/22/202


Validation losses: [0.3559717031625601, 0.3559717031625601, 0.3559717031625601, 0.3559717031625601]


Epoch: 100%|██████████| 4/4 [00:40<00:00, 10.09s/it]


In [44]:
test_data = finbert.get_data('test')


In [45]:
results = finbert.evaluate(examples=test_data, model=trained_model)


04/22/2021 22:15:45 - INFO - utils -   *** Example ***
04/22/2021 22:15:45 - INFO - utils -   guid: test-1
04/22/2021 22:15:45 - INFO - utils -   tokens: [CLS] the bristol port company has sealed a one million pound contract with cooper specialised handling to supply it with four 45 - ton ##ne , custom ##ised reach stack ##ers from ko ##ne ##cr ##ane ##s [SEP]
04/22/2021 22:15:45 - INFO - utils -   input_ids: 101 1996 7067 3417 2194 2038 10203 1037 2028 2454 9044 3206 2007 6201 17009 8304 2000 4425 2009 2007 2176 3429 1011 10228 2638 1010 7661 5084 3362 9991 2545 2013 12849 2638 26775 7231 2015 102 0 0 0 0 0 0 0 0 0 0
04/22/2021 22:15:45 - INFO - utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
04/22/2021 22:15:45 - INFO - utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/22/2021 22:15:45 - INFO - utils -   label: positive (id = 0)
04/22/20

In [47]:

def report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Loss:{0:.2f}".format(loss))
    print("Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    print(classification_report(df[cols[0]], df[cols[1]]))

In [48]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))


In [49]:
report(results,cols=['labels','prediction','predictions'])


Loss:0.40
Accuracy:0.82

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.82      0.78       267
           1       0.71      0.91      0.79       128
           2       0.91      0.81      0.85       575

    accuracy                           0.82       970
   macro avg       0.79      0.85      0.81       970
weighted avg       0.84      0.82      0.83       970



In [65]:

text = "Jim- J Block was just a gas contract. At Enron, risk is controlled through one of two mechanisms: a book \
structure with approved limits and daily reporting of P&L or through a one off approval of a Dash. No risk should \
be taken on outside of these boundaries. Even though there is no capital, a gas contract contains market and \
credit risk and should be approved, providing it doesn't fit into a book. Rick"

In [66]:
model = AutoModelForSequenceClassification.from_pretrained(cl_path, cache_dir=None, num_labels=3)

In [67]:

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/drdata/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [68]:
result = predict(text,model)


04/22/2021 23:46:06 - INFO - utils -   *** Example ***
04/22/2021 23:46:06 - INFO - utils -   guid: 0
04/22/2021 23:46:06 - INFO - utils -   tokens: [CLS] jim - j block was just a gas contract . [SEP]
04/22/2021 23:46:06 - INFO - utils -   input_ids: 101 3958 1011 1046 3796 2001 2074 1037 3806 3206 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/22/2021 23:46:06 - INFO - utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/22/2021 23:46:06 - INFO - utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/22/2021 23:46:06 - INFO - utils -   label: None (id = 9090)
04/22/2021 23:46:06 - INFO - root -   tensor([[-1.4834, -0.7447,  1.9405],
        [-1.4328, -2.0062,  2.3411],
        [-0.9042, -2.0118,  2.1591],
        [

In [69]:
blob = TextBlob(text)
result['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]
result

,sentence,logit,prediction,sentiment_score,textblob_prediction
0,Jim- J Block was just a gas contract.,"[0.029600972, 0.061961032, 0.90843797]",neutral,-0.032360,0.0
1,"At Enron, risk is controlled through one of two mechanisms: a book structure with approved limits and daily reporting of P&L or through a one off approval of a Dash.","[0.022166556, 0.01249324, 0.96534026]",neutral,0.009673,0.0
2,No risk should be taken on outside of these boundaries.,"[0.04399851, 0.014534588, 0.9414669]",neutral,0.029464,0.0
3,"Even though there is no capital, a gas contract contains market and credit risk and should be approved, providing it doesn't fit into a book.","[0.06787374, 0.07114844, 0.8609778]",neutral,-0.003275,0.4
4,Rick,"[0.13935319, 0.10946537, 0.7511814]",neutral,0.029888,0.0


In [70]:
print(f'Average sentiment is %.2f.' % (result.sentiment_score.mean()))


Average sentiment is 0.01.
